In [ ]:
!pip install -q transformers accelerate git+https://github.com/huggingface/diffusers compel

In [ ]:
from diffusers import DiffusionPipeline
import torch
from compel import Compel, ReturnedEmbeddingsType
from diffusers import DiffusionPipeline, AutoencoderKL
from PIL import Image
from datetime import datetime
import os

vae = AutoencoderKL.from_pretrained("madebyollin/sdxl-vae-fp16-fix", torch_dtype=torch.float16)
base = DiffusionPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-base-1.0",
    vae=vae, torch_dtype=torch.float16, variant="fp16",
    use_safetensors=True
)

_ = base.to("cuda")

refiner = DiffusionPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-refiner-1.0",
    text_encoder_2=base.text_encoder_2,
    vae=base.vae,
    torch_dtype=torch.float16,
    use_safetensors=True,
    variant="fp16",
)

_ = refiner.to("cuda")

compel_base = Compel(tokenizer=[base.tokenizer, base.tokenizer_2] , text_encoder=[base.text_encoder, base.text_encoder_2], returned_embeddings_type=ReturnedEmbeddingsType.PENULTIMATE_HIDDEN_STATES_NON_NORMALIZED, requires_pooled=[False, True])
compel_refiner = Compel(tokenizer=refiner.tokenizer_2 , text_encoder=refiner.text_encoder_2, returned_embeddings_type=ReturnedEmbeddingsType.PENULTIMATE_HIDDEN_STATES_NON_NORMALIZED, requires_pooled=True)

In [ ]:
prompt_keywords = [
    "low quality",
    "bad hands",
    "random text",
    "error",
    "missing or extra fingers",
    "cropped",
    "jpeg artifacts",
    "signature",
    "watermark",
    "blurry",
    "ugly",
    "duplicate",
    "mutilated"
]

high_noise_frac = 0.8

In [ ]:
if not os.path.exists("wrong_images"):
    os.mkdir("wrong_images")

i = 0
for keyword in prompt_keywords:
    prompt = ", ".join(prompt_keywords)
    prompt = prompt.replace(keyword, f"({keyword})++++")
    print(prompt)

    conditioning, pooled = compel_base(prompt)
    conditioning_ref, pooled_ref = compel_refiner(prompt)

    for cfg in [8, 9, 10, 11, 12, 13]:
        latents = base(prompt_embeds=conditioning,
            pooled_prompt_embeds=pooled,
            guidance_scale=cfg,
            denoising_end=high_noise_frac,
            output_type="latent")["images"]

        images = refiner(
            prompt_embeds=conditioning_ref,
            pooled_prompt_embeds=pooled_ref,
            guidance_scale=cfg,
            denoising_start=high_noise_frac,
            image=latents,
            ).images

        for image in images:
            image.save(os.path.join("wrong_images", f"{i:03d}.png"))
            i += 1
        

## MIT License

Copyright (c) 2023 Max Woolf

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.